In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import places_api_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vestmannaeyjar,IS,2022-04-14 23:20:25,63.4427,-20.2734,47.70,97,100,24.16
1,1,Upernavik,GL,2022-04-14 23:20:26,72.7868,-56.1549,-12.93,99,37,8.34
2,2,Ribeira Grande,PT,2022-04-14 23:20:26,38.5167,-28.7000,61.20,88,75,23.02
3,3,Igarka,RU,2022-04-14 23:20:27,67.4667,86.5833,32.29,97,100,17.20
4,4,Chuy,UY,2022-04-14 23:20:27,-33.6971,-53.4616,54.34,69,0,9.19


In [4]:
city_data_df = city_data_df[(city_data_df["Max Temp"]>0)]

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=places_api_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
wind = city_data_df["Wind Speed"]
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Igarapava,BR,2022-04-14 23:20:27,-20.0383,-47.7469,76.12,91,25,2.86
6,6,San Cristobal,VE,2022-04-14 23:20:28,7.7669,-72.2250,77.18,66,99,3.00
12,12,Padang,ID,2022-04-14 23:20:29,-0.9492,100.3543,75.25,87,97,2.46
14,14,Hilo,US,2022-04-14 23:18:43,19.7297,-155.0900,83.28,75,75,11.50
17,17,Rikitea,PF,2022-04-14 23:20:30,-23.1203,-134.9692,78.04,68,22,15.70
25,25,Hithadhoo,MV,2022-04-14 23:20:33,-0.6000,73.0833,83.26,71,98,6.44
26,26,Adrar,MR,2022-04-14 23:20:33,20.5022,-10.0711,74.23,23,0,10.29
28,28,Sur,OM,2022-04-14 23:20:34,22.5667,59.5289,85.53,39,22,11.61
29,29,Danshui,CN,2022-04-14 23:20:34,22.7984,114.4672,71.71,71,31,7.34
30,30,Grand Gaube,MU,2022-04-14 23:20:34,-20.0064,57.6608,78.39,76,28,5.99


In [10]:
preferred_cities_df.count()

City_ID       211
City          211
Country       211
Date          211
Lat           211
Lng           211
Max Temp      211
Humidity      211
Cloudiness    211
Wind Speed    211
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Igarapava,BR,76.12,-20.0383,-47.7469,
6,San Cristobal,VE,77.18,7.7669,-72.2250,
12,Padang,ID,75.25,-0.9492,100.3543,
14,Hilo,US,83.28,19.7297,-155.0900,
17,Rikitea,PF,78.04,-23.1203,-134.9692,
25,Hithadhoo,MV,83.26,-0.6000,73.0833,
26,Adrar,MR,74.23,20.5022,-10.0711,
28,Sur,OM,85.53,22.5667,59.5289,
29,Danshui,CN,71.71,22.7984,114.4672,
30,Grand Gaube,MU,78.39,-20.0064,57.6608,


In [12]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import places_api_key

In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": places_api_key,
    "location": "48.8566, 2.3522"}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -16.2259575, 'lng': 39.9057937},
    'viewport': {'northeast': {'lat': -16.22464971970849,
      'lng': 39.90708848029149},
     'southwest': {'lat': -16.2273476802915, 'lng': 39.90439051970849}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name': 'Guest House HESADA',
   'photos': [{'height': 2322,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108692098884708211150">Hetul Sacicant</a>'],
     'photo_reference': 'Aap_uEAa1mZ_mBXYC9_6N5fm9DxOlZSUWIGL5T4rwzj0qfwVKshu27SDoX56zeWwJgquaLF-RSL7gNEeeC9u8pAhZb_uszI8so3j3uwaweAUyHi2y5EyFqMhQCNW0bj8k4oSz91ozEs2wo0hxYqQSEm18i08WJ814rOdwwk0GiOIffb15h4k',
     'width': 4128}],
   'place_id': 'ChIJIZl6DjHhyRgR2A77Y4jlMYg',
   'rating'

In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Igarapava,BR,76.12,-20.0383,-47.7469,Ângelo Palace Hotel Igarapava
6,San Cristobal,VE,77.18,7.7669,-72.2250,Pirineos
12,Padang,ID,75.25,-0.9492,100.3543,Mercure Padang
14,Hilo,US,83.28,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Rikitea,PF,78.04,-23.1203,-134.9692,People ThankYou
25,Hithadhoo,MV,83.26,-0.6000,73.0833,Scoop Guest House
26,Adrar,MR,74.23,20.5022,-10.0711,
28,Sur,OM,85.53,22.5667,59.5289,Sur Plaza Hotel
29,Danshui,CN,71.71,22.7984,114.4672,Grand Skylight International Hotel Huizhou
30,Grand Gaube,MU,78.39,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa


In [16]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))